In [169]:
import numpy as np

In [284]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from time import sleep
from tqdm import tqdm
from IPython.display import clear_output



PROXY = "45.79.230.234:80" 

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

web = webdriver.Chrome(ChromeDriverManager().install())


[WDM] - Current google-chrome version is 98.0.4758
[WDM] - Get LATEST driver version for 98.0.4758


[WDM] - Driver [/Users/yasha_ten/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [285]:
web.get('https://www.oddsportal.com/soccer/belgium/jupiler-pro-league/st-truiden-leuven-xrk8ujnt/')

In [273]:
def return_coef(page_source):
    soup = bs(page_source)
    text = soup.find('span', class_='spc spc-nowrap')
    if text:
        text = text.text
        if 'Opening odds:' in text:
            coef = text.split('Opening odds:')[-1]
            if 'Click' in coef:
                coef = coef.split('Click')[0]

            coef = coef.split(' ')[-1]

            if coef:
                return float(coef)
            else:
                return np.nan
                print('SMTH WITH COEF')
        else:
            return np.nan
            print('No phrase in text')
    else:
        return -1

In [64]:
from selenium.webdriver.common.action_chains import ActionChains

In [85]:
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

def wait_visibility_class_name(web, class_name, timeout=10):
        WebDriverWait(web, timeout).until(
            EC.visibility_of_element_located((By.CLASS_NAME, class_name)))

In [274]:
def return_coefs(href):
    web.get(href)
    wait_visibility_class_name(web, 'table-main.detail-odds.sortable')
    books = web.find_element_by_class_name('table-main.detail-odds.sortable')
    all_books = books.find_elements_by_class_name('lo.odd') + books.find_elements_by_class_name('lo.even')    
    coefs = []
    book_found = 0
    for book in all_books:
        name = book.find_element_by_class_name('name').text
        if name == 'bet365':
            els = book.find_elements_by_css_selector('td')
            for el in els:
                if 'right' in el.get_attribute('class'):
                    action = ActionChains(web)  
                    action.move_to_element(el).perform()
                    sleep(0.5)
                    page_source = web.page_source
                    coef = return_coef(page_source)
                    if coef != -1:
                        coefs.append(coef)
                    else:
                        print('SMTH WITH HTML')
            book_found = 1
            break


    if not book_found:
        print('No bet365 in this list')
        return []
        

    return coefs

In [250]:
source = bs(web.page_source, 'html.parser')

In [272]:
return_coef(web.page_source)

AttributeError: 'NoneType' object has no attribute 'text'

In [258]:
if web.find_elements_by_id('onetrust-accept-btn-handler') != []:
    web.find_element_by_id('onetrust-accept-btn-handler').click()
    sleep(0.5)

In [313]:
data = pd.DataFrame(
    {
        'href': [], 'year': [],
        'month': [], 'day': [],
        'day_of_week': [], 'time': [],
        'team1': [], 'team2': [],
        'score': [], 'k_team1': [],
        'k_draw': [], 'k_team2': []
    }
).set_index('href')



In [314]:
data.to_excel('project/odds.xlsx')

In [226]:
data = pd.read_excel('odds.xlsx', index_col='href')

In [228]:
data = pd.read_excel('odds.xlsx', index_col='href')

links = set()
with open('links.txt', 'r') as f:
    for link in f.readlines():
        links.add(link.strip())
f.close()

for link in links:
    hrefs = get_hrefs(link)
    parse_next_matches(hrefs)
    parse_history(link + 'results')

No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list


In [286]:
def get_date():
    table = web.find_element_by_id('col-content')
    info = table.find_element_by_css_selector('p').text
    date, time = info.split(', ')[1:]
    return date, time

In [327]:
date, time = get_date()
pd.to_datetime(date).strftime('%H')

'00'

In [211]:
def get_hrefs(league_link):
    hrefs = set()
    web.get(league_link)
    wait_visibility_class_name(web, 'table-main')
    table = web.find_element_by_class_name('table-main')
    for line in table.find_elements_by_class_name('name.table-participant'):
        subels = line.find_elements_by_css_selector('a')
        for el in subels:
            href = el.get_attribute('href')
            if 'https' in el.get_attribute('href') and 'inplay' not in el.get_attribute('href'):
                hrefs.add(href)

    return hrefs

def get_team_names():
    table = web.find_element_by_id('col-content')
    title = table.find_element_by_css_selector('h1').text
    team1, team2 = title.split(' - ')
    return team1, team2

def get_date():
    table = web.find_element_by_id('col-content')
    info = table.find_element_by_css_selector('p').text
    date, time = info.split(', ')[1:]
    return date, time

def get_result():
    if web.find_elements_by_class_name('result') != []:
        wait_visibility_class_name(web, 'result')
        text = web.find_element_by_class_name('result').text
        result = text.split(' ')[2]
        return result
    return ''

                
def parse_next_matches(hrefs):
    for href in hrefs:
        if href not in data.index:
            coefs = return_coefs(href)
            team1, team2 = get_team_names()
            date, time = get_date()
            datetime = pd.to_datetime(date)
            year = datetime.year
            month = datetime.strftime('%b')
            day = datetime.day
            day_of_week = datetime.strftime('%A')
            
            if len(coefs) == 3:
                data.loc[href] = [year, month,
                                  day, day_of_week,
                                  time, team1,
                                  team2, np.nan, coefs[0],
                                 coefs[1], coefs[2]]
            elif len(coefs) > 0:
                print('Not enough odds in match: {0}'.format(href))
#             print(date, time)
#             print(team1, team2)
        data.to_excel('odds.xlsx')
            

def parse_history(link):
    hrefs = get_hrefs(link)
    for href in hrefs:
        if href in data.index and data.loc[href, 'score'] != np.nan:
            web.get(href)
            result = get_result()
            data.loc[href, 'score'] = result
            data.to_excel('odds.xlsx')
            


In [216]:
link = 'https://www.oddsportal.com/soccer/england/premier-league/'
hrefs = get_hrefs(link)
parse_next_matches(hrefs)
parse_history(link + 'results')

No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list
No bet365 in this list


In [183]:
link = 'https://www.oddsportal.com/soccer/india/isl/'

In [184]:
web.get(link)

In [189]:
hrefs = get_hrefs(link)

In [206]:
def get_result():
    if web.find_elements_by_class_name('result') != []:
        wait_visibility_class_name(web, 'result')
        text = web.find_element_by_class_name('result').text
        result = text.split(' ')[2]
        return result
    return ''

In [229]:
data

,date,team1,team2,score,k_team1,k_draw,k_team2
href,,,,,,,
https://www.oddsportal.com/soccer/argentina/primera-nacional/deportivo-maipu-ind-rivadavia-KlPLoB85/,2022-02-13 20:00:00,Deportivo Maipu,Ind. Rivadavia,1:1,2.75,2.75,2.60
https://www.oddsportal.com/soccer/argentina/primera-nacional/almirante-brown-chacarita-juniors-fRybo2EM/,2022-02-13 00:30:00,Almirante Brown,Chacarita Juniors,4:2,1.61,3.40,5.00
https://www.oddsportal.com/soccer/argentina/primera-nacional/estudiantes-rio-cuarto-instituto-n3u2RtNj/,2022-02-14 00:15:00,Estudiantes Rio Cuarto,Instituto,NaN,2.40,2.80,2.90
https://www.oddsportal.com/soccer/argentina/primera-nacional/all-boys-atletico-atlanta-YB9K23qc/,2022-02-13 00:15:00,All Boys,Atletico Atlanta,1:0,2.30,2.62,3.40
https://www.oddsportal.com/soccer/argentina/primera-nacional/gimnasia-jujuy-brown-adrogue-GzwfnraG/,2022-02-13 00:00:00,Gimnasia Jujuy,Brown Adrogue,3:3,2.45,2.62,3.10
https://www.oddsportal.com/soccer/argentina/primera-nacional/ca-belgrano-de-cordoba-atl-rafaela-Q9STutxp/,2022-02-15 00:30:00,Belgrano,Atl. Rafaela,NaN,1.75,3.50,4.50
https://www.oddsportal.com/soccer/argentina/primera-nacional/club-atletico-guemes-ca-estudiantes-K8LxvK6d/,2022-02-13 23:00:00,Club A. Guemes,CA Estudiantes,NaN,2.25,3.00,3.00
https://www.oddsportal.com/soccer/argentina/primera-nacional/gimnasia-mendoza-san-martin-s-j-C28O1Nb3/,2022-02-13 20:30:00,Gimnasia Mendoza,San Martin S.J.,NaN,2.10,3.00,3.30
https://www.oddsportal.com/soccer/england/premier-league/west-ham-newcastle-utd-2uDrbXU2/,2022-02-19 12:30:00,West Ham,Newcastle,NaN,1.57,4.00,5.50
